# Batch run of the environment model 
for higher quality results

## Batch run

In [ ]:
import seaborn as sns
from schema import *
from typing import Dict, Any, Tuple
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# if iterables: tries all possible combinations of the parameters
# parameters: Dict = {
#     "width": range(8, 15),
#     "height": range(8, 15),
#     "n_survivors": range(1, 11),
#     "n_save_zones": range(1, 11),
#     "n_robot_agents": range(1, 4),
#     "seed": 42,
# }

parameters: Dict = {
    "width": [8, 10, 12],
    "height": [8, 10, 12],
    "n_survivors": range(1, 11),
    "n_save_zones": range(1, 11),
    "n_robot_agents": [1, 2, 4],
    "seed": 42,
}

# calculate the number of iterations for all parameters
count = 1
for key, value in parameters.items():
    # value can be a list or a range
    if isinstance(value, range):
        count *= len(list(value))
    elif isinstance(value, list):
        count *= len(value)
print("Count of combinations:", count)

In [ ]:
# if true -> run batch
# if false -> load pickle file from output folder
run_batch = False

In [ ]:
if run_batch:
    batch_results = mesa.batch_run(
        model_cls=EnvironmentModel,
        parameters=parameters,
        iterations=2, # for each possible combination of parameters
        number_processes=None,
        data_collection_period=-1,
        display_progress=True,
        max_steps=100,
    )
    # auto multiprocessing if number_processes is None

In [ ]:
PICKLE_FILE = "./output/batch_results.pkl"
df: pd.DataFrame = None

if run_batch:
    df = pd.DataFrame(batch_results)
    pd.DataFrame.to_pickle(df, PICKLE_FILE)

else:
    # load the results from the pickle file
    try:
        df = pd.DataFrame(pd.read_pickle(PICKLE_FILE))
    except:
        print("batch results not available, file not found, etc.")

print(f"The results have {len(df)} rows.")
print(f"The results have {len(df.columns)} columns.")
print("Columns:")
print(df.columns)
df.head()

## Data transformation

In [ ]:
# drop cols
df.drop(
    [
        "seed", 
        "Survivors", 
        "SaveZones", 
        "MazeWidth", 
        "MazeHeight", 
        "AllSurvivorsRescued", 
        "Tile", 
        "StillRunning",
        "AgentID",
        "TransportedSurvivor",
    ],
    axis=1,
    inplace=True,
)
print(f"The results have reduced {len(df.columns)} columns.")
print(df.columns)

# get the ratio of TilesMoved per SurvivorCount as the column "TilesMovedPerSurvivor"
df["TilesMovedPerSurvivor"] = df.apply(
    lambda x: x["TilesMoved"] / x["n_survivors"] if x["n_survivors"] > 0 else 0, axis=1
)

# get the ratio of TilesMoved per SaveZoneCount as the column "TilesMovedPerSaveZone"
df["TilesMovedPerSaveZone"] = df.apply(
    lambda x: x["TilesMoved"] / x["n_save_zones"] if x["n_save_zones"] > 0 else 0,
    axis=1,
)

# get pathlengths parameters. only works if data_collection_period is set to 1
df["PathlengthMin"] = df.apply(
    lambda x: EnvironmentModel.get_min_pathlength(
        x["InitialPathlengths"]
    ),
    axis=1,
)
df["PathlengthMax"] = df.apply(
    lambda x: EnvironmentModel.get_max_pathlength(x["InitialPathlengths"]),
    axis=1,
)
df["PathlengthMean"] = df.apply(
    lambda x: EnvironmentModel.get_mean_pathlength(x["InitialPathlengths"]),
    axis=1,
)

In [ ]:
# sns.scatterplot(data=results_df, x="SurvivorCount", y="TilesMoved")
sns.scatterplot(data=df, x="n_save_zones", y="TilesMoved")

## Data analysis

In [ ]:
# plot as a histogram
# sns.boxenplot(data=results_df, x="SurvivorCount", y="TilesMovedPerSurvivor")
sns.boxenplot(data=df, x="n_save_zones", y="TilesMoved")

### Correlation matrix of chosen cols

In [ ]:
# choose some columns
print(df.columns)
df_corr = df.reset_index()
df_corr = df_corr[
    [
        "width",
        "height",
        "MeanWallDensity",
        "n_survivors",
        "n_save_zones",
        "n_robot_agents",
        "PathlengthMin",
        "PathlengthMax",
        "PathlengthMean",
        "Step",
        "TotalTilesMoved",
        # "TilesMovedPerSurvivor",
        # "TilesMovedPerSaveZone",
    ]
]
df_corr.rename(
    columns={
        "width": "Maze width",
        "height": "Maze height",
        "MeanWallDensity": "Walls per tile",
        "n_survivors": "Survivors",
        "n_save_zones": "Savezones",
        "n_robot_agents": "Robots",
        "PathlengthMin": "Min pathlength",
        "PathlengthMax": "Max pathlength",
        "PathlengthMean": "Avg pathlength",
        "Step": "Total steps",
        "TotalTilesMoved": "Tiles moved",
    },
    inplace=True,
)

corr = df_corr.corr()

# Correlation matrix with mask for upper triangle as matplotlib figure
sns.set_theme(style="white")
mask = np.triu(np.ones_like(corr, dtype=bool))

f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(
    corr,
    mask=mask,
    cmap=cmap,
    vmax=1,
    vmin=-1,
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
    annot=True,
    fmt=".2f",
)
plt.title("Correlation matrix")
plt.xticks(rotation=75, ha="right")
plt.savefig("./output/correlation_heatmap.png", dpi=300, bbox_inches="tight")

### 